# ¿Cómo se relacionan los rendimientos y la volatilidad para las acciones de energía y el mercado en general?

## Introducción

**Contexto empresarial.** Usted es analista de un gran banco centrado en inversiones en acciones de recursos naturales. Recientemente, realizó un análisis de las siguientes acciones de energía y cómo su volumen de operaciones está relacionado con su volatilidad:

1. Dominion Energy Inc. (Stock Symbol: D)
2. Exelon Corp. (Stock Symbol: EXC)
3. NextEra Energy Inc. (Stock Symbol: NEE)
4. Southern Co. (Stock Symbol: SO)
5. Duke Energy Corp. (Stock Symbol: DUK)

Su jefe quiere que realice un análisis para poder averiguar cómo dimensionar las posiciones potenciales en estas acciones ... es decir, qué porcentaje de la cartera de inversiones debería dedicarse a cada una de estas acciones. Específicamente, quiere que observe los rendimientos diarios y la volatilidad de cada acción, así como del mercado en general (es decir, no solo del sector energético).

Esto es importante porque una alta volatilidad implica un mayor riesgo, y a su jefe le gustaría saber si los rendimientos potenciales de estas acciones de energía de alta volatilidad lo compensan por el riesgo adicional. Además, debido a que su desempeño se mide o se compara con el mercado en general, quiere comprender si estas acciones generalmente superan al mercado en general.

**Problema de negocio.** Basado en el contexto anterior, su jefe le ha hecho la siguiente pregunta: **"¿Cuál es la relación entre la volatilidad diaria y los rendimientos de estas acciones y cuál es la relación entre los rendimientos diarios de estas acciones? y el mercado de valores en general?"**

**Contexto analítico.** Los datos que le han proporcionado están en formato de valores separados por comas (CSV) y comprenden datos de precios y volumen de operaciones para las acciones anteriores. Procederá mediante: (1) realizar una limpieza preliminar de los datos; (2) crear características adicionales necesarias para nuestro análisis; (3) etiquetar los datos en grupos de volatilidad, o regímenes, y determinar cómo la volatilidad se relaciona con los rendimientos; y finalmente (4) comparar estos rendimientos con los del mercado en general.

In [1]:
# Import libraries required for this case: pandas, numpy, seaborn


## Limpieza de datos preliminar

Antes de que podamos continuar con el análisis y el modelado de datos, primero debemos determinar si los datos relevantes son adecuados para continuar tal como están o si necesitan una limpieza adicional. En este caso, hemos recibido un archivo de valores separados por comas (CSV) que incluye los siguientes datos:

1. **Date:** The day of the year
2. **Open:** The stock opening price of the day
3. **High:** The highest observed stock price of the day
4. **Low:** The lowest observed stock price of the day
5. **Close:** The stock closing price of the day
6. **Adj Close:** The adjusted stock closing price for the day (adjusted for splits and dividends)
7. **Volume:** The volume of the stock traded over the day
8. **Symbol:** The symbol for that particular stock

Un problema muy común que surge en los conjuntos de datos son los valores perdidos. Veamos cómo identificar si nuestro conjunto de datos tiene o no este problema y cómo lidiar con los valores faltantes.

In [2]:
# Load and view head of DataFrame
raw_df = pd.read_csv('EnergySectorData.csv')
raw_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D


Comencemos por determinar si tenemos valores perdidos. Podemos usar el método ```isnull()``` de DataFrame en pandas para verificar los valores de ```NaN``` en raw_df (es decir, verificar los valores nulos):

In [4]:
# Check if there are missing values (NaNs)
raw_df.isnull().sum()

Date          0
Open          2
High         14
Low           7
Close         7
Adj Close     7
Volume       22
Symbol        0
dtype: int64

Aquí vemos que tenemos algunos valores perdidos. En su lugar, usemos el método ``mean()`` para determinar qué porcentaje de cada columna nos falta.

In [4]:
raw_df.isnull().mean()

Date         0.000000
Open         0.000319
High         0.002231
Low          0.001116
Close        0.001116
Adj Close    0.001116
Volume       0.003506
Symbol       0.000000
dtype: float64

Vemos aquí que nos falta menos del **X,X%** de las observaciones en una columna determinada.



### Ejercicio 1:
No queremos que falten valores en nuestro análisis. ¿Cuál de las siguientes opciones es la PEOR opción sobre cómo proceder en este caso?

(a) Completar el valor faltante de cualquier día con el valor del día anterior

(b) Reemplazar los valores faltantes volviendo a recopilar los datos

(c) Estimar los valores faltantes interpolando los valores de otros puntos de datos similares

(d) Eliminar filas del conjunto de datos que contienen valores faltantes

Proceder utilizando el enfoque definido

## Estandarización de fechas

Nos gustaría poder analizar estas acciones juntas a lo largo del tiempo. Esto sería más fácil si todas las acciones no contuvieran datos faltantes para el mismo conjunto de fechas. Primero averigüemos si este es el caso. Una forma de hacer esto es usar el método ```groupby``` para agrupar por ```Date```, luego usar la función ```count ()``` para enumerar cuántas fechas distintas tenemos. Dado que hay un total de 1259 filas por símbolo, debe haber un recuento de 1259 para cada símbolo.

In [8]:
# How many data rows do we have for each Symbol
progress_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1230,1230,1230,1230,1230,1230,1230
DUK,1249,1249,1249,1249,1249,1249,1249
EXC,1239,1239,1239,1239,1239,1239,1239
NEE,1251,1251,1251,1251,1251,1251,1251
SO,1259,1259,1259,1259,1259,1259,1259


Dado que la mayoría de los símbolos no tienen un recuento de 1259 para la columna ``Date``, claramente hay algunos valores inconsistentes. Algunos de estos duplicados tendrán valores perdidos (NaN), así que enumerémoslos de nuevo primero:

In [9]:
# Check if there are missing values (NaNs)
raw_df.isnull().sum()

Date          0
Open          2
High         14
Low           7
Close         7
Adj Close     7
Volume       22
Symbol        0
dtype: int64

In [8]:
# Drop the missing values
progress_df = raw_df.dropna().copy()

In [9]:
# How many data rows do we have for each Symbol
progress_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1230,1230,1230,1230,1230,1230,1230
DUK,1249,1249,1249,1249,1249,1249,1249
EXC,1239,1239,1239,1239,1239,1239,1239
NEE,1251,1251,1251,1251,1251,1251,1251
SO,1259,1259,1259,1259,1259,1259,1259


Aún así, vemos que diferentes símbolos tienen diferentes números de fechas. Nos gustaría que todos los símbolos tuvieran el mismo conjunto de fechas para fines de análisis **¿Por qué?**.

Creemos un nuevo ```clean_df``` que corresponda a un DataFrame con el mismo número de filas para cada ```Símbolo```, donde todos los símbolos comparten el mismo conjunto de fechas:

In [10]:
set_dates_D = set(progress_df[progress_df['Symbol'] == 'D']['Date'])
set_dates_EXC = set(progress_df[progress_df['Symbol'] == 'EXC']['Date'])
set_dates_NEE = set(progress_df[progress_df['Symbol'] == 'NEE']['Date'])
set_dates_SO = set(progress_df[progress_df['Symbol'] == 'SO']['Date'])
set_dates_DUK = set(progress_df[progress_df['Symbol'] == 'DUK']['Date'])
set_unique_dates = set.intersection(set_dates_D,set_dates_EXC,set_dates_NEE,set_dates_SO,set_dates_DUK)

In [11]:
# Filter new DataFrame for only the dates that are present in every symbol (i.e. the overlapping dates)
clean_df = progress_df[progress_df['Date'].isin(set_unique_dates)].copy()

In [12]:
# Si todo salió bien cada símbolo debería estar asociado con 1192 fechas.
clean_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1192,1192,1192,1192,1192,1192,1192
DUK,1192,1192,1192,1192,1192,1192,1192
EXC,1192,1192,1192,1192,1192,1192,1192
NEE,1192,1192,1192,1192,1192,1192,1192
SO,1192,1192,1192,1192,1192,1192,1192


Ahora vemos que cada símbolo tiene el mismo número de fechas únicas. Escribamos un programa de verificación rápida para asegurarnos de que el resultado ``clean_df`` tenga las mismas fechas para cada símbolo.

### Ejercicio 2:
Escriba código para asegurarse de que cada uno de los símbolos comparta el mismo conjunto de fechas únicas. (Sugerencia: use el método ``set ()``.)

Ahora que hemos completado la limpieza preliminar de los datos, avancemos con la determinación de las relaciones entre: (1) retornos de acciones y volatilidad, y (2) retornos de acciones y retornos de mercado más amplios.

## Agregar variables adicionales requeridas para nuestro análisis

Recuerde que la pregunta original requiere que investiguemos tanto los rendimientos diarios de las acciones como la volatilidad de esos rendimientos. Esto significa que importantes medidas de interés son:

1. Daily (open to close) stock return (Return) --> (Close / Open) - 1
2. Volatility of daily stock return (VolStat) --> (High - Low) / Open

¿Por qué son importantes cada uno de estos?

1. Volatilidad: da una idea de la cantidad de movimiento de precios en un día determinado. La volatilidad está directamente relacionada con el nivel de riesgo involucrado en la tenencia de acciones.
2. Retorno: nos da una idea del retorno de la inversión durante un período de tiempo.

Adicionalmente, creemos una nueva variable llamada Volume_Millions.

Calculemos estas estadísticas y agréguelas al DataFrame ```clean_df```:

Aquí vemos que hemos agregado tres columnas a ```clean_df```, llamadas, ```VolStat```, ```Return``` y ```Volume_Millions``` (la última es solo por conveniencia, ya que los valores de la columna ``Volumen`` son bastante grandes).

Dado que estamos buscando analizar la relación entre la volatilidad diaria y los rendimientos, una columna adicional que tiene sentido agregar es una que dice ``True`` cuando el rendimiento diario es positivo y ``Falso`` cuando el Retorno diario negativo. Luego, podemos agrupar los días en cohortes de rendimiento positivo y negativo y comparar la volatilidad promedio en esos días. Nombraremos esta columna ```ReturnFlag```.

Podemos lograr esto usando una **función anónima**; es decir, una función que está definida pero no nombrada:
```python
lambda argumentos: expresión
```

La palabra clave ```lambda``` le dice a Python que estamos usando una función anónima. A continuación, los ```argumentos``` son el nombre que damos a las entradas. Puede ser ```x``` o ```y```, o como quiera llamarlo el usuario. En este caso, usaremos el nombre ```fila``` para el nombre del argumento de entrada, ya que la entrada será de hecho una fila de un DataFrame. La ```expresión``` es lo que luego se aplica a los ``` argumentos```; esta es la función.

Echemos un vistazo a cómo podemos usar funciones anónimas para crear la columna ``ReturnFlag```:

In [15]:
clean_df['ReturnFlag'] = clean_df.apply(lambda row: True if row['Return'] > 0 else False, axis=1)
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True


Observe que el método ```apply ()``` toma una función anónima y la aplica a las filas del DataFrame mediante el uso del segundo argumento ```axis```. ```axis = 0``` aplica la función a las columnas, mientras que ``axis = 1`` aplica la función a las filas.

Entonces, ¿qué está sucediendo en la siguiente declaración?
```python
clean_df ['ReturnFlag'] = clean_df.apply (lambda row: True if row ['Return]> 0 else False, axis = 1)
```

1. ```pandas``` reconoció a través del método ``apply`` que está operando en el DataFrame ```clean_df```
2. El método ```apply``` toma una función como entrada que se aplicará al DataFrame ``` clean_df```
3. Dado que el segundo argumento de ```apply``` es ```axis = 1```, la entrada en la función anónima es una sola fila
4. Para cada fila, ```row ['Return']``` devuelve el valor de ``Return`` para esa fila, y posteriormente se pasa a través de la instrucción if, devolviendo True si es mayor que cero y False de otra manera
5. El nuevo valor se almacena en la columna ``clean_df ['ReturnFlag']``

### Ejercicio 3:

Usando ```apply ()``` y ```lambda```, escriba el código para crear una nueva columna llamada ```YYYY``` en ```clean_df```, donde la nueva columna es el año de la observación como una cadena. Por ejemplo, si el valor de la fila ``Date`` es 2014-07-28, entonces el valor en la nueva columna para el año sería '2014'. Recuerde que puede acceder a los primeros 4 caracteres de alguna cadena ```my_string``` usando ```my_string [: 4]```.

Avancemos con el etiquetado de los regímenes de volatilidad presentes en los datos; estos regímenes son útiles para desglosar el análisis de rendimiento de las acciones por períodos de volatilidad baja, media y alta. Permitirá un análisis más granular que solo mirar los promedios generales sin un desglose.

### Ejercicio 4:
Usando ```apply ()``` y ```lambda```, escriba un script para crear una nueva columna en ``` clean_df``` llamada ```AvgDailyPrice``` que calcula un precio diario promedio basado sobre si el volumen diario supera los 5 millones. Establezca el valor de la nueva columna en (Open + High + Low + Close) / 4 si el volumen es mayor que 5 millones, o establezca el valor en (High + Low + Close) / 3 si el volumen es menor o igual a 5 millones.

## Etiquetado de regímenes de volatilidad del sector energético

Al igual que en el caso 1.2, nos gustaría etiquetar los períodos de alta y baja volatilidad en una nueva columna llamada ``VolLevel`` para cada símbolo, utilizando algunos valores de límite inferior y superior. Por ejemplo, en el caso del Símbolo D nos gustaría tener una nueva columna con valor determinado por:

```python
if VolStrat > upper_threshold_dict['D']:
    VolLevel = '3_HIGH'
elif VolStrat < lower_threshold_dict['D']:
    VolLevel = '1_LOW'
else:
    VolLevel = '2_MEDIUM'
```

Es decir, este etiquetado debe aplicarse a cada fila y los valores de umbral deben corresponder al símbolo de esa fila. Agruparemos por esta columna y veremos si podemos encontrar nuevos conocimientos en los datos.

In [16]:
# Determine lower bounds (we choose to use 25th percentile)
lower_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.25).to_dict() # 25th percentile bound
lower_threshold_dict

{'D': 0.010240046986389077,
 'DUK': 0.010018315803797114,
 'EXC': 0.011881680089172456,
 'NEE': 0.010258642787424582,
 'SO': 0.009734019893739423}

In [17]:
# Determine upper bounds (we choose to use 75th percentile)
upper_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.75).to_dict() # 75th percentile bound
upper_threshold_dict

{'D': 0.017960914526108228,
 'DUK': 0.017598380774085175,
 'EXC': 0.021801523265676366,
 'NEE': 0.017680218024250814,
 'SO': 0.016830447068579304}

Nuevamente, nuestro objetivo es etiquetar los períodos de volatilidad baja, media y alta. Definamos una nueva columna llamada ``VolLevel`` para cada símbolo usando algunos valores de límite superior e inferior. Definamos una función personalizada que se aplicará a cada fila para lograr este objetivo.

In [18]:
# Our custom function, input is a row from the agg_df, and the output is a string, either LOW, MEDIUM, or HIGH
def my_custom_row_function(row):
    row_symbol = row['Symbol']    # the Symbol value in the row
    row_volstat = row['VolStat']  # the VolStat value in the row
    
    lower_threshold = lower_threshold_dict[row_symbol] # Dictionary of {string:float}
    upper_threshold = upper_threshold_dict[row_symbol] # Dictionary of {string:float}
    
    # The function decision, return value depending on low, medium, or high volatility
    if row_volstat > upper_threshold:
        return '3_HIGH'
    elif row_volstat < lower_threshold:
        return '1_LOW'
    else:
        return '2_MEDIUM'

Ahora apliquemos la función a cada fila del DataFrame ```clean_df``` usando la función ``` lambda```. Almacenamos los valores devueltos en la nueva columna ```VolLevel```:

In [19]:
# Apply my_custom_row_function to the Pandas DataFrame, row by row (axis=1)
clean_df['VolLevel'] = clean_df.apply(lambda row: my_custom_row_function(row), axis=1)
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,VolLevel
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,3_HIGH
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2_MEDIUM
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,3_HIGH
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,3_HIGH
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2_MEDIUM


Si bien el flujo de trabajo aquí puede parecer complejo al principio, la capacidad de aplicar funciones personalizadas, agrupar por ciertas características y construir estadísticas de resumen resultará invaluable a medida que avanza hacia análisis más avanzados.

### Ejercicio 5:

Usando ```clean_df``` y una declaración ```lambda``` dentro de ```apply ()```, escriba una función ```new_custom_function ()``` y un script para agregar una nueva columna a el DataFrame (llámelo ``EnhancedVolLevel``) que opera de manera similar a VolLevel, pero en su lugar da cinco categorías de niveles de volatilidad usando la siguiente lógica para determinar la etiqueta para el nivel de volatilidad:

```python
if VolStrat > 90th percentile:
    VolLevel = '5_VERY_HIGH'
elif VolStrat > 75th percentile:
    VolLevel = '4_HIGH'
elif VolStrat > 25th percentile:
    VolLevel = '3_MEDIUM'
elif VolStrat > 10th percentile:
    VolLevel = '2_LOW'
else:
    VolLevel = '1_VERY_LOW'
```

Recuerde que cada percentil debe calcularse mediante un símbolo. Utilice estas nuevas etiquetas para ver si existen patrones entre los niveles de volatilidad y la dirección de los retornos. Produzca el DataFrame para poder ejecutar el siguiente comando:

```python
clean_df.groupby(['Symbol','EnhancedVolLevel'])['ReturnFlag'].mean()
```

Vemos que la volatilidad y la rentabilidad de las acciones no muestran patrones sólidos en términos de la dirección de la rentabilidad media (positiva o negativa) para un régimen de volatilidad dado.

## Comparación de rendimientos de acciones con rendimientos de mercado más amplios

Ahora, analicemos la segunda parte de la pregunta de su jefe: ¿cuál es la relación entre los rendimientos más amplios del mercado y los rendimientos de estas cinco acciones de energía? El índice S&P 500 es un índice bursátil compuesto por unas 500 empresas públicas estadounidenses de gran capitalización. El índice se utiliza a menudo como representación del mercado de valores de EE. UU. Si podemos determinar si existe o no una fuerte relación entre los rendimientos de estas 5 acciones de energía y los del índice S&P 500, podemos determinar si hay características idiosincrásicas significativas en juego entre los rendimientos de las acciones del sector energético, o si los rendimientos son impulsado únicamente por el mercado en general.

Los rendimientos de mercado para el ETF (fondo cotizado en bolsa) del índice S&P 500 negociable están disponibles en ``SPY.csv`` (el símbolo de "acciones" del ETF es SPY). Carguemos los datos y agreguemos los rendimientos diarios a los datos del sector de energía limpia:

In [21]:
# Load file into DataFrame
market_df = pd.read_csv('SPY.csv')

In [22]:
market_df['Symbol'] = 'SPY' # add column for symbol
market_df['Return'] = (market_df['Close'] / market_df['Open']) - 1.0 # calculate return
market_df['VolStat'] = (market_df['High'] - market_df['Low']) / market_df['Open']
market_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Return,VolStat
0,2014-08-18,196.800003,197.449997,196.690002,197.360001,178.729111,75424000,SPY,0.002846,0.003862
1,2014-08-19,197.839996,198.539993,197.440002,198.389999,179.661896,59135000,SPY,0.002780,0.005560
2,2014-08-20,198.119995,199.160004,198.080002,198.919998,180.141846,72763000,SPY,0.004038,0.005451
3,2014-08-21,199.089996,199.759995,198.929993,199.500000,180.667160,67791000,SPY,0.002059,0.004169
4,2014-08-22,199.339996,199.690002,198.740005,199.190002,180.386368,76107000,SPY,-0.000752,0.004766


Nos gustaría fusionar los rendimientos del mercado en ``market_df`` y los datos de stock de energía en ``clean_df``. Esto se puede lograr usando ``pd.merge()`` - un método versátil para unir DataFrames.

Para aquellos de ustedes familiarizados con SQL, la combinación y unión de DataFrames se puede lograr de la misma manera que SQL realiza estas tareas. En este caso, nos gustaría usar la intersección de fechas en las fechas ```clean_df``` y ``market_df`` como índices en la fusión (es decir, en lenguaje SQL, realizaremos un ``inner``):

In [23]:
# Merge inner (merge market_df onto clean_df using the dates of clean_df as the keys)
merged_df = pd.merge(clean_df, market_df[['Date','Return']], how='inner', on='Date', suffixes=('','_SPY'))

In [24]:
# Check how many dates are in the intersection
merged_df.groupby('Symbol')['Date'].count()

Symbol
D      1178
DUK    1178
EXC    1178
NEE    1178
SO     1178
Name: Date, dtype: int64

In [25]:
merged_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,VolLevel,EnhancedVolLevel,Return_SPY
0,2014-08-18,68.970001,69.250000,68.559998,68.680000,56.164867,1375600.0,D,0.010004,-0.004205,1.3756,False,1_LOW,2_LOW,0.002846
1,2014-08-18,31.930000,32.099998,31.730000,31.820000,26.593485,4037400.0,EXC,0.011588,-0.003445,4.0374,False,1_LOW,2_LOW,0.002846
2,2014-08-18,96.589996,97.180000,95.889999,96.139999,82.166077,1098200.0,NEE,0.013355,-0.004659,1.0982,False,2_MEDIUM,3_MEDIUM,0.002846
3,2014-08-18,43.529999,43.720001,43.279999,43.380001,34.210857,2979800.0,SO,0.010108,-0.003446,2.9798,False,2_MEDIUM,3_MEDIUM,0.002846
4,2014-08-18,72.300003,72.650002,71.919998,72.089996,58.080738,1826000.0,DUK,0.010097,-0.002905,1.8260,False,2_MEDIUM,3_MEDIUM,0.002846


### Ejercicio 6:

Usando solo el símbolo D en ```clean_df``` y ``market_df``, use ```pd.merge ()``` para determinar cuántas fechas hay en ```clean_df``` que no están en ```market_df```. Además, ¿cuántas fechas hay en ```market_df``` que no están en ```clean_df```? (Sugerencia: isnull () puede ser útil para simplificar la solución).

### Ejercicio 7:

Use ```market_df``` y ``clean_df`` para crear un nuevo DataFrame ```modified_clean_df``` que es igual que ``clean_df`` pero con una nueva columna llamada ``MeanMonthSPYReturn``. Cada valor en la nueva columna debe ser el rendimiento mensual medio de SPY para el mes dado por cada fila de ``Date``. La salida para la primera y última fila del DataFrame ``modified_clean_df`` debe producir la siguiente salida de head (1) y tail (1):

```python
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].head(1))
```

                 Date       Symbol YYYYMM MeanMonthSPYReturn
                 2014-08-01 D      201408 0.001443

```python
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].tail(1))
```
                 Date       Symbol YYYYMM MeanMonthSPYReturn
                 2019-07-26 DUK    201907 0.000167

## Desglose de los rendimientos según el rendimiento general del mercado

Ahora podemos comenzar nuestro análisis granular de cómo los retornos de mercado más amplios están relacionados con los retornos de acciones individuales.

Comencemos por dividir los rendimientos de mercado más amplios en cuantiles. Este enfoque de análisis con cuantiles que tomaremos se emplea comúnmente en el análisis de datos para determinar cómo la magnitud de una variable se relaciona con otra variable de interés.

Podemos explorar esta idea con el método ``pd.qcut ()``. Es decir, ``pd.qcut()`` nos permitirá recortar los rendimientos del mercado por cuantiles (eventualmente los agruparemos por cuantiles) y, por lo tanto, nos permitirá calcular estadísticas resumidas (como el rendimiento promedio) para cada cuantil.

Primero extraigamos los retornos usando la conveniencia del método ``pivot()`` en un DataFrame. El pivote de un DataFrame se puede lograr especificando:

1. Un índice sobre el que pivotar. En este caso elegimos ``Date``.
2. Columnas que nos gustaría tener después del pivote. En este caso, nos gustaría columnas que sean el ``Símbolo``.
3. Los valores que mostrará cada par (fila, columna). En este caso nos gustaría tener el ``Return``.

Usaremos pivot con ``merged_df`` utilizando estas entradas de parámetros para generar un DataFrame donde las filas son las Fechas, cada columna es el Símbolo y los valores son el retorno diario de apertura a cierre para la fecha y el símbolo dados.

In [26]:
# Extract returns from merged_df, where we use pivot to simplify the task
return_df = merged_df.pivot(index='Date', columns='Symbol', values='Return')
return_df.head()

Symbol,D,DUK,EXC,NEE,SO
Date,,,,,
2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446
2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741
2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508
2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000
2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195


Combinemos los rendimientos de mercado más amplios del SPY ```market_df``` cargado anteriormente:

In [27]:
# Let's merge the SPY (broader market) returns by Date onto the return_df DataFrame
full_df = pd.merge(return_df, market_df[['Date','Return']].set_index('Date'), left_index=True, right_index=True)
full_df = full_df.rename(columns={'Return':'MarketReturn'}).reset_index()
full_df.head()

,Date,D,DUK,EXC,NEE,SO,MarketReturn
0,2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446,0.002846
1,2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741,0.002780
2,2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508,0.004038
3,2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000,0.002059
4,2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195,-0.000752


A través de unas pocas líneas simples hemos creado un DataFrame ```full_df``` donde cada valor es un retorno diario de apertura a cierre, ya sea para uno de los cinco símbolos en estudio o para el mercado más amplio.

Procedemos utilizando ``pd.qcut ()`` con 10 cuantiles. En general, el número de cuantiles debe elegirse en función del nivel de detalle de la vista que necesite.

In [28]:
# Create 10 quantile categories by the market return
num_quantiles = 10
full_df['market_quantile'] = pd.qcut(full_df['MarketReturn'],num_quantiles,labels=False)
full_df.head()

,Date,D,DUK,EXC,NEE,SO,MarketReturn,market_quantile
0,2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446,0.002846,7
1,2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741,0.002780,7
2,2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508,0.004038,7
3,2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000,0.002059,6
4,2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195,-0.000752,3


Ahora agrupemos por ```market_quantile``` y calculemos el rendimiento medio de todos los símbolos:

In [29]:
# Group by market quantile and calculate the mean return
full_df.groupby('market_quantile').mean()

,D,DUK,EXC,NEE,SO,MarketReturn
market_quantile,,,,,,
0,-0.004553,-0.003282,-0.006711,-0.004723,-0.002890,-0.013363
1,-0.002423,-0.001444,-0.002686,-0.001001,-0.000830,-0.005045
2,-0.001612,-0.000617,-0.001087,-0.000693,-0.000061,-0.002577
3,0.001010,0.000420,-0.001277,0.000728,0.000938,-0.001154
4,0.001189,0.000615,0.000377,0.000903,0.000444,-0.000113
5,-0.001516,-0.001656,-0.000699,-0.000985,-0.001240,0.000887
6,0.001950,0.001233,0.002246,0.002210,0.001040,0.002055
7,0.001804,0.000505,0.001006,0.000831,0.000963,0.003568
8,0.002148,0.002366,0.004323,0.003042,0.002563,0.005787


Cada valor en la salida de DataFrame anterior es una media de los retornos diarios para un símbolo dado, donde la media se toma en todas las fechas que corresponden al ``market_quantile`` listado como el índice del DataFrame de salida. Tenga en cuenta que los cuantiles más altos indican rendimientos de mercado más altos, mientras que los números de cuantiles más bajos indican rendimientos de mercado más bajos (en este caso, rendimientos de mercado negativos).

Vemos aquí que los rendimientos de las acciones de energía de hecho siguen un patrón cuando los rendimientos de SPY son grandes (número de cuantil más alto) o pequeños (número de cuantil más bajo). Es decir, los rendimientos de las acciones de las acciones individuales siguen el mismo patrón que los del mercado en general. Por tanto, el mercado más amplio tiene un efecto sobre los rendimientos de una sola acción; es decir, la magnitud del rendimiento general del mercado de valores está correlacionada con la magnitud del rendimiento de las acciones individuales.

### Ejercicio 8:

Creamos cuantiles para los retornos de mercado y posteriormente calculamos el retorno medio para cada uno de estos cuantiles. Realice un análisis similar al anterior, pero en su lugar agrupe por cuantiles para la volatilidad del mercado en lugar de los rendimientos del mercado, y calcule la desviación estándar de los rendimientos para cada categoría de cuantiles de volatilidad del mercado en lugar del rendimiento medio.

De manera similar a cómo las direcciones de los rendimientos de las acciones siguen las del mercado en general, la volatilidad de los rendimientos de las acciones también sigue la volatilidad de los rendimientos del mercado en general. Esto justifica un análisis más detallado de la causa raíz de este efecto como un proyecto futuro.

## Conclusiones

Hemos explorado los rendimientos de las acciones de las cinco acciones del sector energético en términos de sus propios regímenes de volatilidad y sus rendimientos y volatilidad en relación con el mercado en general. Descubrimos que para las acciones en estudio, no existe un vínculo fuerte entre el nivel de volatilidad y la dirección del rendimiento diario de las acciones. Además, encontramos que cuando se comparan las acciones con el mercado en general, sus rendimientos y niveles de volatilidad se amplifican cuando los rendimientos del mercado y los niveles de volatilidad son altos. Estos hallazgos indican que existe un vínculo intrínseco entre los rendimientos y la volatilidad, tanto en el caso de una sola acción como en el caso del mercado en general. Esto brinda una variedad de vías de exploración para proyectos de seguimiento.

## Para rescatar

En este caso, aprendió varias herramientas de manipulación de datos en ``pandas``, que incluyen funciones anónimas, agrupación, fusión, corte de cuantiles y pivoteo, mientras utiliza las técnicas de análisis de agregación y transformación de datos que hemos aprendido anteriormente.

``` pandas``` es un paquete increíblemente versátil y puede aumentar significativamente la productividad y brindar información empresarial excepcional. Estas técnicas deben servir como una base sólida para cualquier análisis de datos futuro que pueda realizar.